# By ChatGPT

In [1]:
from ebooklib import epub, ITEM_DOCUMENT  # 수정된 부분
from bs4 import BeautifulSoup
import html2text

def epub_to_markdown(epub_path, output_md_path):
    # EPUB 파일 읽기
    book = epub.read_epub(epub_path)
    
    # 변환기 설정
    converter = html2text.HTML2Text()
    converter.ignore_links = False  # 링크 유지
    converter.ignore_images = False  # 이미지 유지
    converter.ignore_tables = False  # 표 유지
    
    md_content = ""

    # EPUB 파일에서 본문 추출
    for item in book.get_items():
        if item.get_type() == ITEM_DOCUMENT:  # 수정된 부분
            # HTML 파싱
            soup = BeautifulSoup(item.content, "html.parser")
            
            # 제목 추출 (h1~h6 태그 변환)
            for header in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"]):
                header.string = "#" * int(header.name[1]) + " " + (header.get_text() or "")
                
            # HTML -> Markdown 변환
            md_content += converter.handle(str(soup)) + "\n\n"
    
    # 결과 저장
    with open(output_md_path, "w", encoding="utf-8") as f:
        f.write(md_content)
    
    print(f"Markdown 파일이 생성되었습니다: {output_md_path}")


In [2]:
# 사용 예시
book_file = "documents/The Collected Works of Spinoza.epub"
epub_to_markdown(book_file, "The Collected Works of Spinoza.qmd")


/Users/skcho/miniconda3/envs/translation/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/skcho/miniconda3/envs/translation/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Markdown 파일이 생성되었습니다: The Collected Works of Spinoza.qmd


# By DeepSeek

In [3]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import html2text
import os
import datetime

def epub_to_markdown_ds(epub_path, output_file = None):
    # EPUB 파일 읽기
    book = epub.read_epub(epub_path)
    
    # HTML to Markdown 변환기 설정
    h = html2text.HTML2Text()
    h.ignore_links = False
    h.body_width = 0  # 줄 바꿈 비활성화
    
    markdown_content = []
    
    # 스파인 순서대로 아이템 처리
    items = {item.id: item for item in book.get_items()}
    
    for item_id, _ in book.spine:
        item = items.get(item_id)
        if item and item.get_type() == ebooklib.ITEM_DOCUMENT:
            # HTML 내용 파싱
            soup = BeautifulSoup(item.get_content(), 'html.parser')
            
            # 본문 내용 추출
            body = soup.find('body')
            if body:
                # HTML을 Markdown으로 변환
                text = h.handle(str(body))
                markdown_content.append(text)
    
    # output_file 지정하지 않을 경우, epub_path의 이름을 사용하고 현재 날짜와 시간을 추가하여 저장
    if not output_file:
        base_name = os.path.basename(epub_path)
        name, _ = os.path.splitext(base_name)
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{name}_{current_time}.md"

    # Markdown 파일 저장
    with open(output_file, 'w', encoding='utf-8') as md_file:
        md_file.write('\n\n'.join(markdown_content))

In [4]:
input_epub = "Counterfactuals/Counterfactuals and Causal Inference.epub"

# output_md = "output_by_deepseek.md"
epub_to_markdown_ds(input_epub)
print(f"Markdown 파일 생성 완료")

Markdown 파일 생성 완료


In [4]:
input_epub = "The Compassionate Mind/Paul_Gilbert_The_Compassionate_Mind.epub"
output_md = "output_by_deepseek2.md"
epub_to_markdown_ds(input_epub, output_md)
print(f"Markdown 파일 생성 완료: {output_md}")

/Users/skcho/miniconda3/envs/translation/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/skcho/miniconda3/envs/translation/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Markdown 파일 생성 완료: output_by_deepseek2.md


# By Claude

In [5]:
from ebooklib import epub
import html2text
import os

def extract_epub_to_markdown(epub_path, output_dir):
    """
    EPUB 파일을 구조를 유지하며 Markdown으로 변환하는 함수
    """
    # EPUB 파일 로드
    book = epub.read_epub(epub_path)
    
    # Markdown 변환기 설정
    h = html2text.HTML2Text()
    h.body_width = 0  # 줄바꿈 비활성화
    h.emphasis_mark = '*'
    h.strong_mark = '**'
    h.ignore_images = False
    h.ignore_links = False
    h.ignore_tables = False
    
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # 메타데이터 추출 및 저장
    metadata_md = "# 도서 정보\n\n"
    
    # 제목 처리
    titles = book.get_metadata('DC', 'title')
    if titles:
        metadata_md += f"- **제목**: {titles[0][0]}\n"
    
    # 작성자 처리
    creators = book.get_metadata('DC', 'creator')
    if creators:
        metadata_md += f"- **작성자**: {creators[0][0]}\n"
    
    # 언어 처리
    languages = book.get_metadata('DC', 'language')
    if languages:
        metadata_md += f"- **언어**: {languages[0][0]}\n"
    
    # 식별자 처리
    identifiers = book.get_metadata('DC', 'identifier')
    if identifiers:
        metadata_md += f"- **식별자**: {identifiers[0][0]}\n"
    
    # 출판사 처리
    publishers = book.get_metadata('DC', 'publisher')
    if publishers:
        metadata_md += f"- **출판사**: {publishers[0][0]}\n"
    
    with open(os.path.join(output_dir, "000_metadata.md"), "w", encoding="utf-8") as f:
        f.write(metadata_md)
    
    # 목차 정보 추출
    toc = book.toc
    def process_toc(toc_items, level=1):
        toc_md = ""
        for item in toc_items:
            if isinstance(item, tuple):
                # 섹션 제목과 링크
                section, href = item
                toc_md += f"{'  ' * (level-1)}- [{section}]({href})\n"
            elif isinstance(item, list):
                # 하위 목차 처리
                toc_md += process_toc(item, level+1)
        return toc_md
    
    toc_md = "# 목차\n\n" + process_toc(toc)
    with open(os.path.join(output_dir, "001_toc.md"), "w", encoding="utf-8") as f:
        f.write(toc_md)
    
    # 컨텐츠 추출 및 변환
    for i, item in enumerate(book.get_items(), 2):
        # 문서 타입 확인 방법 변경
        if isinstance(item, epub.EpubHtml):  # XHTML 문서인지 확인
            # 파일명 생성
            filename = f"{i:03d}_{item.get_name().replace('.html', '.md')}"
            
            # HTML 컨텐츠를 Markdown으로 변환
            html_content = item.get_content().decode('utf-8')
            md_content = h.handle(html_content)
            
            # Markdown 파일 저장
            with open(os.path.join(output_dir, filename), "w", encoding="utf-8") as f:
                f.write(md_content)

In [6]:
# 사용 예시
epub_file = "Counterfactuals/Counterfactuals and Causal Inference.epub"
output_directory = "book_markdown"

if __name__ == "__main__":
    # epub_file = "book.epub"
    # output_directory = "book_markdown"
    
    try:
        extract_epub_to_markdown(epub_file, output_directory)
        print(f"변환이 완료되었습니다. 결과물이 {output_directory} 디렉토리에 저장되었습니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")

변환이 완료되었습니다. 결과물이 book_markdown 디렉토리에 저장되었습니다.
